<a href="https://colab.research.google.com/github/vishnunagar109/Machine_Learning/blob/main/Sentiment_Analysis/Amz_review_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import bz2
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torchtext.legacy.data import Field,TabularDataset,BucketIterator
import spacy

In [ ]:
root_dir = 'gdrive/MyDrive/Datasets/torch_text_data/Amazon_sentiment_analysis'

In [ ]:
# # # #Data preparation in next 4 cells
# #This is a one time task, after saving the data into csv files, we only need to load csv files

# #1. Saving the data into dictionaries in order to convert them into dataframes later
# # # #

# train_data = {'text':[],'score':[]}
# test_data = {'text':[],'score':[]}
# i = 0
# with bz2.open(os.path.join(root_dir,'train.ft.txt.bz2'), "rt") as bz_file:
#     for line in bz_file:
#       if(i>=400000):
#         break
#       i = i+1
#       text = line[11:]
#       label = int(line[9])
#       train_data['text'].append(text)
#       train_data['score'].append(label)

# i = 0
# with bz2.open(os.path.join(root_dir,'test.ft.txt.bz2'), "rt") as bz_file:
#     for line in bz_file:
#       if(i>=10000):
#         break
#       i = i+1
#       text = line[11:]
#       label = int(line[9])
#       test_data['text'].append(text)
#       test_data['score'].append(label)

In [ ]:
# print(len(train_data['text']))
# print(len(test_data['text']))

In [ ]:
# ####2. Converting the dictionaries into pandas dataframes #####

# df_train = pd.DataFrame(train_data,columns=['text','score'])
# df_test = pd.DataFrame(test_data,columns=['text','score'])

In [ ]:
# ####3. Varifying the converted data #####

# print(df_train.head())
# print(df_test.head())

In [ ]:
# ####Export the dataframes into csv files #####

# df_train.to_csv(os.path.join(root_dir,'train1.csv'),index=False)
# df_test.to_csv(os.path.join(root_dir,'test1.csv'),index=False)
# df_train.to_json(os.path.join(root_dir,'train1.json'),orient='records',lines=True)
# df_test.to_json(os.path.join(root_dir,'test1.json'),orient='records',lines=True)

In [ ]:
# !zip "gdrive/MyDrive/Datasets/torch_text_data/Amazon_sentiment_analysis/test.zip" "gdrive/MyDrive/Datasets/torch_text_data/Amazon_sentiment_analysis/test.json"

In [ ]:
# train_data = pd.read_csv(os.path.join(root_dir,'train.csv'),index_col=0)
# test_data = pd.read_csv(os.path.join(root_dir,'test.csv'),index_col=0)

In [ ]:
# !python -m spacy download en
spacy_eng = spacy.load('en')

def tokenizer_eng(text):
  return [tok.text for tok in spacy_eng.tokenizer(text)]

In [ ]:
text = Field(sequential=True,use_vocab=True,tokenize=tokenizer_eng)
score = Field(sequential=False,use_vocab=False)
fields = {'text':('t',text),'score':('s',score)}
train_data,test_data = TabularDataset.splits(path=root_dir,train='train1.json',test='test1.json',format='json',fields=fields)


In [ ]:
text.build_vocab(train_data,max_size=10000,min_freq=2)

In [ ]:
print(train_data[0].__dict__.keys())
print(train_data[0].__dict__.values())

dict_keys(['t', 's'])
dict_values([['Stuning', 'even', 'for', 'the', 'non', '-', 'gamer', ':', 'This', 'sound', 'track', 'was', 'beautiful', '!', 'It', 'paints', 'the', 'senery', 'in', 'your', 'mind', 'so', 'well', 'I', 'would', 'recomend', 'it', 'even', 'to', 'people', 'who', 'hate', 'vid', '.', 'game', 'music', '!', 'I', 'have', 'played', 'the', 'game', 'Chrono', 'Cross', 'but', 'out', 'of', 'all', 'of', 'the', 'games', 'I', 'have', 'ever', 'played', 'it', 'has', 'the', 'best', 'music', '!', 'It', 'backs', 'away', 'from', 'crude', 'keyboarding', 'and', 'takes', 'a', 'fresher', 'step', 'with', 'grate', 'guitars', 'and', 'soulful', 'orchestras', '.', 'It', 'would', 'impress', 'anyone', 'who', 'cares', 'to', 'listen', '!', '^_^'], 2])


In [ ]:
#Hyperparameters
input_size = len(text.vocab)
output_size = 1
hidden_size = 1024
emb_size = 200
batch_size = 64
num_epochs = 50
lr = 0.01
num_layers = 1

In [ ]:
#Model
class Network(nn.Module):
  def __init__(self,input_size,emb_size,hidden_size,output_size,num_layers):
    super(Network,self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.num_layers = num_layers
    self.embs = nn.Embedding(input_size,emb_size)
    self.gru = nn.GRU(emb_size,hidden_size,bidirectional=True)
    self.fc = nn.Linear(2*hidden_size,output_size)
    self.sigmoid = nn.Sigmoid()

  def forward(self,x,hidden):
    x = self.embs(x)
    # print(x.size())
    # print(x)
    # print(x)
    out,hidden = self.gru(x,hidden)
    # print(out.size())
    # out = out.reshape(out.shape[1],-1)
    # print(out.size()) 
    # print(out.size(0))
    out = out[int(out.size(0))-1,:,:]
    # out1 = out[-1][:][:]
    # out2 = out[:][:][-1]
    # print(out1.size())
    # print(out2.size())
    # print(out.size())
    out = self.sigmoid(self.fc(out))
    # print(out)
    # print(out.size())

    return out

  def init_hidden(self):
    return torch.zeros(2*self.num_layers,batch_size,self.hidden_size)

Model = Network(input_size,emb_size,hidden_size,output_size,num_layers)

In [ ]:
#Model Hyperparameters
pad_idx = text.vocab.stoi['<pad>']
criterion = nn.BCELoss()
# criterion1 = nn.BCELoss()
optimizer = optim.Adam(params=Model.parameters(),lr=lr)
lambda1 = lambda epoch: 0.65 ** epoch
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)

In [ ]:
#Getting the batches
train_iter,test_iter = BucketIterator.splits((train_data,test_data),batch_size=batch_size,shuffle=True)

In [ ]:
#training
i = 0
for epoch in range(num_epochs):
  i = 0
  epoch_loss = 0
  # hidden = Model.init_hidden()
  for idx,batch in enumerate(train_iter):
    x = batch.t
    y = batch.s
    hidden = Model.init_hidden()

    # print(x.size())
    # print(x.size())
    # print(y)
    y = torch.tensor([0 if val==1 else 1 for val in y])
  #   # print(y.size())
    # print(y)
    output = Model(x,hidden)
  #   # print(y)
  #   # print(output.size())
  #   # print(torch.argmax(output,1))
  #   # output = torch.argmax(output,1)
    # print(output.squeeze())
  #   # print(y.squeeze())
  #   # y = y.squeeze()
    optimizer.zero_grad()
    loss = criterion(output.squeeze().float(),y.float())
    loss.backward()
    optimizer.step()
    epoch_loss+=loss
    if(i%100==0):
      print(loss)
    i+=1

  scheduler.step()
  print(f"epoch:{epoch+1}|loss:{float(epoch_loss)/idx}")


tensor(0.6956, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6951, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6747, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6959, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.7641, grad_fn=<BinaryCrossEntropyBackward>)
